[Data description](http://www.opendatanetwork.com/dataset/data.cityofnewyork.us/9w7m-hzhe) 

Articles

* [Lower fines](http://www.nydailynews.com/new-york/restaurants-cheer-plan-slash-penalties-article-1.1430407) (nice window photo)
* [Grading pooh-pooh](http://www.nydailynews.com/opinion/city-council-throws-rotten-tomatoes-restaurant-grades-article-1.1036545) (grading placards)

# To do
1. Violations '7' (obstruction) and 
    1. '20E' & 20F (grades not posted)
    1. 10A (no toilet paper)

# Requirements

In [ ]:
from __future__ import division, print_function
import pandas as pd
import numpy as np
import bokeh as bk
import pickle
from bokeh.plotting import figure, output_notebook, show, output_file
from bokeh.io import push_notebook
from bokeh.models import ColumnDataSource, HoverTool, Span, Range1d
from bokeh.embed import components

In [3]:
output_notebook()

Loading BokehJS ...

# Initial manipulation (first time only)

In [19]:
file_name = 'DOHMH_New_York_City_Restaurant_Inspection_Results_new.csv'
with open(file_name) as f:
    column_names = f.readline().splitlines()
    
column_names = column_names[0].split(',')
for x in ['PHONE','VIOLATION DESCRIPTION','RECORD DATE','BUILDING','STREET','CRITICAL FLAG']:
    column_names.remove(x)
    
df = pd.read_csv(file_name,usecols=column_names,parse_dates=['INSPECTION DATE','GRADE DATE'])

In [67]:
dict_action = {0: 'No violations were recorded at the time of this inspection.',
 1: 'Violations were cited in the following area(s).',
 2: 'Establishment Closed by DOHMH.  Violations were cited in the following area(s) and those requiring immediate action were addressed.',
 3: 'Establishment re-opened by DOHMH',
 4: 'Establishment re-closed by DOHMH'}

In [22]:
# uninspected
df_uninspected = df[df['ACTION'].isnull()] # Same as INSPECTION DATE == 1900-01-01
df_uninspected.to_csv('uninspected.csv',index=False)

In [23]:
# inspected
df = df[df['ACTION'].notnull()]
df['ACTION'].replace(dict( (v,k) for k,v in dict_action.items()),inplace=True)
df.to_csv('inspected.csv',index=False)

In [29]:
# 2015 inspected
df15 = df[(df['INSPECTION DATE'] >= pd.to_datetime('2015-01-01')) & (df['INSPECTION DATE'] < pd.to_datetime('2016-01-01'))]
df15.to_csv('inspected_2015.csv',index=False)

In [ ]:
# violation description
violation_df = pd.read_csv('DOHMH_New_York_City_Restaurant_Inspection_Results_new.csv', 
                           usecols=['VIOLATION CODE','VIOLATION DESCRIPTION'])
violation_description = violation_df.groupby(violation_df['VIOLATION CODE'])['VIOLATION DESCRIPTION'].unique()
for i in range(len(violation_description)):
    violation_description.iloc[i] = violation_description.iloc[i][0]

dict_violation = violation_description.to_dict()
for k,v in dict_violation.viewitems():
    if isinstance(v,basestring):
        v = v.decode('utf-8').encode('latin1').decode('utf-8')
        v = v.replace(u'\xba',u'\xb0')
        v = v.replace("''''","'")
        v = v.replace(u'\x1a',"'")
        v = v.replace("\'","'")
        dict_violation[k] = v

In [68]:
with open('dicts.pickle','wb') as f:
    pickle.dump([dict_action,dict_violation], f) 

##### DF of violations plus critical flag (not used). 

violation_df = pd.read_csv('DOHMH_New_York_City_Restaurant_Inspection_Results_new.csv', 
                           usecols=['VIOLATION CODE','VIOLATION DESCRIPTION','CRITICAL FLAG'])
violation_description = violation_df.groupby(['VIOLATION CODE','VIOLATION DESCRIPTION'])['CRITICAL FLAG'].unique()
for i in range(len(violation_description)):
    violation_description.iloc[i] = violation_description.iloc[i][0]

vd = violation_description.reset_index(level=1)
x = vd['VIOLATION DESCRIPTION'].str.decode('utf-8').str.encode('latin1').str.decode('utf-8')
x = x.replace(u'\xba',u'\xb0')
x = x.replace("''''","'")
x = x.replace(u'\x1a',"'")
x = x.replace("\'","'")
vd['VIOLATION DESCRIPTION'] = x
vd.to_csv('violation_codes.csv',encoding='utf-8')

# Start here

In [4]:
df15 = pd.read_csv('inspected_2015.csv',
                   parse_dates=['INSPECTION DATE','GRADE DATE'])
original_file = 'DOHMH_New_York_City_Restaurant_Inspection_Results_new.csv'

In [5]:
with open('dicts.pickle','rb') as f:
    dict_action, dict_violation = pickle.load(f)

    days_of_week = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
title_font = '14pt'
axis_font = '10pt'

In [7]:
def plot_output(p, filename):
    script, div = components(p)
    with open('bokeh_source/{}.html'.format(filename),'w') as f:
        f.write(script)
        f.write(div)

    output_file('images/{}.html'.format(filename))
    show(p)

#### if other years needed

In [4]:
df = pd.read_csv('inspected.csv',parse_dates=['INSPECTION DATE','GRADE DATE'])

# General info

In [32]:
print('critical violations:', sum(x < '08' for x in dict_violation.keys()))
print('general violations:', sum((x > '08') & (x < '11')  for x in dict_violation.keys()) + 1)
# includes '10K' (not in scoresheet) and + 1  for '99B' ('Other general')
print('unscored violations:', sum((x > '15') & (x < '99')  for x in dict_violation.keys()))
print('total:', len(dict_violation))

critical violations: 50
general violations: 18
unscored violations: 30
total: 98


In [11]:
with open(original_file) as f:
    column_names = f.readline().splitlines()
    
column_names

['CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE']

# No of restaurants by year

In [33]:
restaurants_by_year = df.groupby(df['INSPECTION DATE'].map(lambda x:x.year))['CAMIS'].nunique()
restaurants_by_year.index.names = ['year']
restaurants_by_year.columns = ['number']

In [74]:
source = ColumnDataSource(dict(
    year=restaurants_by_year[1:-1].index,
    number=restaurants_by_year[1:-1].values))

p = figure(title='No. of NYC restaurants in dataset by year',
           plot_width = 400,
           plot_height = 400,
           tools='save')
p.add_tools(HoverTool(tooltips='@number'))
p.title_text_font_size = title_font
p.xaxis.axis_label_text_font_size = axis_font


p.line('year','number',source=source,line_width=2)
p.square('year','number',source=source,size=8)

p.xaxis.minor_tick_line_color = None

plot_output(p,'number_by_year')

# 2015

In [64]:
restaurant_number_2015 = df15['CAMIS'].nunique()

## Common violations

In [10]:
df15 = pd.read_csv('inspected_2015.csv',parse_dates=['INSPECTION DATE','GRADE DATE'])
# df15 = df[(df['INSPECTION DATE'] >= pd.to_datetime('2015-01-01')) & (df['INSPECTION DATE'] < pd.to_datetime('2016-01-01'))]

In [65]:
violation_prevalence = df15.groupby('VIOLATION CODE')['CAMIS'].nunique()
violation_prevalence = pd.concat([violation_prevalence,
                                  violation_prevalence*100/restaurant_number_2015,
                                  pd.Series(dict_violation)],
                                 join='inner',axis=1)
violation_prevalence.index.name = 'violation_code'
violation_prevalence.columns = ['number','percentage','description']
violation_prevalence.sort_values('percentage',inplace=True,ascending=False)

In [66]:
from bokeh.models import Range1d
from bokeh.models import NumeralTickFormatter

dft = violation_prevalence[violation_prevalence['percentage']>=10]
dft.loc[:,'order'] = range(len(dft),0,-1) # chart start at 1
dft.loc[:,'half'] = dft['percentage']/2

hover = HoverTool(tooltips=[('code','@violation_code'),
                            ('prevalence','@percentage{1.1}%'),
                            ('description','@description')])

wh = 500
p = figure(title = 'Common violations in 2015',
           plot_width = wh,
           plot_height = wh,
           x_axis_label = 'percent of restaurants inspected',
           y_axis_label = 'violation code',
           tools = 'save',
           x_range = Range1d(0,70), 
           y_range = dft.index.tolist()[::-1])
p.add_tools(hover)

# Critical violations
dftt = dft[dft.index < '08'] # temp df 
critical = ColumnDataSource(dftt.to_dict('list'))
critical.add(dftt.index, name="violation_code")
p.rect('half', 'order', width='percentage', 
       height=0.4,
       fill_color='coral',
       line_color='coral',
       legend='Critical violations',
       source=critical)

# General violations
dftt = dft[dft.index > '08'] # temp df 
general = ColumnDataSource(dftt.to_dict('list'))
general.add(dftt.index, name="violation_code")
p.rect('half', 'order', width='percentage', 
       height=0.4,
       legend='General violations',
       source=general)

p.legend.orientation = 'bottom_right'
p.xaxis[0].formatter = NumeralTickFormatter(format="0")
p.yaxis.major_tick_line_color = None
p.ygrid.grid_line_color = None

p.title_text_font_size = title_font
p.xaxis.axis_label_text_font_size = axis_font
p.yaxis.axis_label_text_font_size = axis_font


plot_output(p,'violations')


C:\Anaconda\lib\site-packages\pandas\core\indexing.py:288: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Anaconda\lib\site-packages\pandas\core\indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Anaconda\lib\site-packages\bokeh\core\properties.py:714: BokehDeprecationWarning: bokeh.models.annotations.orientation was deprecated in Bokeh 0.11; please use Legend.location instead
  super(HasProps, self).__setattr__(name, value)


# Score distribution

In [14]:
df_prepermit = df15[df15['INSPECTION TYPE'] == 'Pre-permit (Operational) / Initial Inspection'].groupby(
                  ['INSPECTION DATE','CAMIS'])['SCORE'].unique()
df_prepermit = df_prepermit.map(lambda x: x[0])
score_dist = df_prepermit.value_counts()

import mf2 as mf
mf.score_distribution_plot(score_dist,'pre-permit')

NameError: global name 'title_font' is not defined

In [1]:
import my_functions as mf
# import mf2 as mf

mf.hello()

hello


## Initial inspection

In [137]:
len(df15[df15['INSPECTION TYPE'] == 'Pre-permit (Operational) / Initial Inspection'])

7665

In [87]:
df_initial = df15[df15['INSPECTION TYPE']
                  =='Cycle Inspection / Initial Inspection'].groupby(
                  ['INSPECTION DATE','CAMIS'])['SCORE'].unique()
df_initial = df_initial.map(lambda x: x[0])

In [125]:
score_distribution = df_initial.value_counts()
score_distribution.sort_index(inplace=True)

print('scores < 0:', score_distribution[score_distribution.index<0].sum())
score_distribution = score_distribution[score_distribution.index>=0]
n = score_distribution.sum()
print('valid scores:', n)

scores < 0: 0
valid scores: 27110


In [126]:
def score_distribution_plot(score_distribution,title):
    n = score_distribution.sum()

    score_distribution = score_distribution.to_frame(name='freq')
    score_distribution['half'] = score_distribution['freq']/2
    score_distribution['percent'] = score_distribution['freq']/n

    dt = score_distribution[score_distribution.index < 13.5] # A 
    A_source = ColumnDataSource(dt.to_dict('list'))
    A_source.add(dt.index,name='score')

    dt = score_distribution[(score_distribution.index > 13.5) & (score_distribution.index < 27.5)] # B 
    B_source = ColumnDataSource(dt.to_dict('list'))
    B_source.add(dt.index,name='score')

    dt = score_distribution[score_distribution.index > 27.5] # C 
    C_source = ColumnDataSource(dt.to_dict('list'))
    C_source.add(dt.index,name='score')

    wh = 500
    p = figure(title=title,
              x_axis_label='score',
              y_axis_label='count',
              plot_width = wh,
              plot_height = wh,
              x_range = Range1d(-3,120),
              y_range = Range1d(0,2700))
    hover = HoverTool(tooltips=[('score','@score'),
                                ('count','@freq'),
                                ('percent','@percent{0.0%}')])
    p.add_tools(hover)

    # h_color = 'deepskyblue'
    # h_color = 'lightskyblue'

    def histo(p,source,color,grade):
        tally = sum(source.data['freq'])
        p.rect('score','half',
               width=1,
               height='freq',
               fill_color=color,
               line_color=color,
               hover_fill_color=color,
               hover_line_color='#333333',
               legend='{}: {:>3.0f}%'.format(grade, tally*100/n),
               source=source)

    histo(p, A_source,'dodgerblue','A')
    histo(p, B_source,'LimeGreen','B')
    histo(p, C_source,'orange','C')

    p.title_text_font_size = title_font
    p.xaxis.axis_label_text_font_size = axis_font
    p.yaxis.axis_label_text_font_size = axis_font

    plot_output(p,title.replace(' ','_'))

In [127]:
score_distribution_plot(score_distribution,'Initial inspection score distribution')

ERROR:C:\Anaconda\lib\site-packages\bokeh\core\validation\check.pyc:W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure, ViewModel:Plot, ref _id: 712c4513-836c-4932-8edb-2bb7da91aaff
ERROR:C:\Anaconda\lib\site-packages\bokeh\core\validation\check.pyc:W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure, ViewModel:Plot, ref _id: e089e928-f64f-4927-b6a8-20dee84e5ee0
ERROR:C:\Anaconda\lib\site-packages\bokeh\core\validation\check.pyc:W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure, ViewModel:Plot, ref _id: fd71b610-019d-4f49-943c-5a38be4760be
ERROR:C:\Anaconda\lib\site-packages\bokeh\core\validation\check.pyc:W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure, ViewModel:Plot, ref _id: 0ed28312-07cb-41c3-9ea3-6f60cc88aa88
ERROR:C:\Anaconda\lib\site-packages\bokeh\core\validation\check.pyc:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: half [renderer: GlyphRenderer, ViewModel:GlyphRenderer, ref _id: 724e7ce4-02b2-4b51-9ff1-

## Reinspection

In [105]:
df15['INSPECTION TYPE'].unique()
df_reinspect = df15[df15['INSPECTION TYPE']
                  =='Cycle Inspection / Re-inspection'].groupby(
                  ['INSPECTION DATE','CAMIS'])['SCORE'].unique()
df_reinspect = df_reinspect.map(lambda x: x[0])

In [128]:
score_distribution = df_reinspect.value_counts()
score_distribution.sort_index(inplace=True)

In [129]:
print('scores < 0:', score_distribution[score_distribution.index<0].sum())
score_distribution = score_distribution[score_distribution.index>=0]
n = score_distribution.sum()
print('valid scores:', n)

scores < 0: 7
valid scores: 15142


In [130]:
score_distribution_plot(score_distribution,'Re-inspection score distribution')

ERROR:C:\Anaconda\lib\site-packages\bokeh\core\validation\check.pyc:W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure, ViewModel:Plot, ref _id: 712c4513-836c-4932-8edb-2bb7da91aaff
ERROR:C:\Anaconda\lib\site-packages\bokeh\core\validation\check.pyc:W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure, ViewModel:Plot, ref _id: e089e928-f64f-4927-b6a8-20dee84e5ee0
ERROR:C:\Anaconda\lib\site-packages\bokeh\core\validation\check.pyc:W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure, ViewModel:Plot, ref _id: fd71b610-019d-4f49-943c-5a38be4760be
ERROR:C:\Anaconda\lib\site-packages\bokeh\core\validation\check.pyc:W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure, ViewModel:Plot, ref _id: 0ed28312-07cb-41c3-9ea3-6f60cc88aa88
ERROR:C:\Anaconda\lib\site-packages\bokeh\core\validation\check.pyc:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: half [renderer: GlyphRenderer, ViewModel:GlyphRenderer, ref _id: 724e7ce4-02b2-4b51-9ff1-

## Breakdown by cuisine type

In [31]:
df15[df15['SCORE']>27]['CAMIS'].nunique() 

3618

In [43]:
cuisine_c_score =  df15[df15['SCORE']>27].groupby('CUISINE DESCRIPTION')['CAMIS'].nunique() 

In [44]:
cuisine_c_score.sort_values(ascending=False, inplace=True)

In [45]:
cuisine_number_2015 = df15.groupby('CUISINE DESCRIPTION')['CAMIS'].nunique() 

In [46]:
cuisine_number_2015.sort_values(ascending=False).head()

CUISINE DESCRIPTION
American            5569
Chinese             2178
CafÃ©/Coffee/Tea    1233
Pizza               1066
Italian              968
Name: CAMIS, dtype: int64

In [47]:
cuisine_2015 = pd.concat([
        cuisine_number_2015, cuisine_number_2015*100/cuisine_number_2015.sum(), 
        cuisine_c_score, cuisine_c_score*100/cuisine_c_score.sum() 
    ], axis=1)

In [50]:
cuisine_2015.columns = ['number','number_%','c_number','c_number_%']

In [54]:
cuisine_2015.sort_values(by='number',ascending=False,inplace=True)

In [56]:
cuisine_2015.fillna(0,inplace=True)

In [58]:
cuisine_2015['c_number'] = cuisine_2015['c_number'].apply(int)

In [59]:
cuisine_2015

,number,number_%,c_number,c_number_%
American,5569,24.475893,694,19.181868
Chinese,2178,9.572364,458,12.658928
CafÃ©/Coffee/Tea,1233,5.419066,127,3.510227
Pizza,1066,4.685096,163,4.505252
Italian,968,4.254384,136,3.758983
"Latin (Cuban, Dominican, Puerto Rican, South & Central American)",829,3.643476,185,5.113322
Mexican,747,3.283084,153,4.228856
Japanese,702,3.085307,142,3.924820
Bakery,672,2.953457,109,3.012714
Caribbean,610,2.680965,123,3.399668


In [18]:
df15[df15['ACTION']==2].head(10) 

,CAMIS,DBA,BORO,ZIPCODE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,INSPECTION TYPE
402,50002803,EL NUEVO TENAMPA RESTAURANTE,BRONX,10458,Mexican,2015-10-01,2,04L,Critical,66.0,NaN,NaT,Cycle Inspection / Initial Inspection
822,41361879,SPICE,MANHATTAN,10003,Thai,2015-05-04,2,08A,Not Critical,53.0,NaN,NaT,Cycle Inspection / Initial Inspection
837,41686787,SPICE SYMPHONY,MANHATTAN,10016,Indian,2015-01-02,2,08A,Not Critical,43.0,NaN,NaT,Cycle Inspection / Initial Inspection
997,50018037,CANTON LOUNGE,MANHATTAN,10013,Chinese,2015-12-21,2,10F,Not Critical,55.0,NaN,NaT,Cycle Inspection / Re-inspection
1261,41705732,BENTON CAFE,MANHATTAN,10017,Japanese,2015-10-23,2,20F,Not Critical,NaN,NaN,NaT,Administrative Miscellaneous / Initial Inspection
1262,41643867,DUNKIN' DONUTS,QUEENS,11373,Donuts,2015-07-13,2,04N,Critical,48.0,NaN,NaT,Cycle Inspection / Re-inspection
1388,41722476,RIVER STYX,BROOKLYN,11222,American,2015-08-15,2,06A,Critical,70.0,NaN,NaT,Cycle Inspection / Re-inspection
1422,50011580,KODAMA SUSHI,MANHATTAN,10036,Japanese,2015-02-02,2,08A,Not Critical,44.0,NaN,NaT,Cycle Inspection / Initial Inspection
1523,50033323,SPICE,QUEENS,11418,Caribbean,2015-06-12,2,02B,Critical,56.0,NaN,NaT,Pre-permit (Operational) / Initial Inspection
1629,41563537,BENITO ONE,MANHATTAN,10013,Italian,2015-02-10,2,04J,Critical,59.0,NaN,NaT,Cycle Inspection / Re-inspection


In [30]:
dict_violation['04M']

u"Live roaches present in facility's food and/or non-food areas."

In [105]:
df15[df15['SCORE']==2]['CAMIS'].nunique() 

# Day of week

In [358]:
day_2015 = df15.groupby('INSPECTION DATE')['CAMIS'].nunique()  # Count unique inspections

In [359]:
day_2015 = day_2015.reindex(pd.date_range('2015-01-01','2015-12-31'))
day_2015.fillna(0,inplace=True)
day_2015 = day_2015.apply(int)

In [360]:
holidays = pd.to_datetime(['2015-01-01','2015-01-19','2015-02-16','2015-05-25','2015-07-04',
            '2015-09-07','2015-10-12','2015-11-03','2015-11-26','2015-12-25'])
# New Year, MLK, President's, Memorial, ID4, 
# Labor, Columbus, Election, Thanksgiving, Xmas
day_2015.drop(holidays,inplace=True)

In [361]:
day_2015 = day_2015.to_frame('INSPECTIONS')
day_2015['DAY'] = day_2015.index.dayofweek
day_2015.index.names = ['DATE']

In [429]:
p = figure(title='No. of NYC restaurant inspections by day (2015)',
           x_axis_label='*NY state holidays are excluded.',
          x_range=['Mon','Tue','Wed','Thu','Fri','Sat','Sun'],
          y_range=Range1d(0,320))
hover = HoverTool(tooltips='@DATE')
p.add_tools(hover)
p.title_text_font_size = '14pt'
p.xaxis.axis_label_text_font_size = '10pt'

source = ColumnDataSource(day_2015.to_dict("list"))
source.add(day_2015.index.strftime('%m/%d/%y'), name="DATE")

# freq = day_2015.groupby('DAY')['INSPECTIONS'].value_counts() # Size for scatter plot
# no longer used

p.circle(day_2015['DAY']+1,'INSPECTIONS',
         # size = day_2015.apply(lambda x: freq[x['DAY'],x['INSPECTIONS']], axis=1) *2.3, 
         size = 6, alpha=0.5, line_color=None,
         source = source,
         legend = 'Date(s)') # +1 b/c range starts at 0

mean = day_2015.groupby("DAY")['INSPECTIONS'].mean()
std = day_2015.groupby("DAY")['INSPECTIONS'].std()
source2 = ColumnDataSource({})
source2.add(mean.values,name="DATE") 

err_x = []
err_y = []

for x, y, yerr in zip(mean.index+1,mean.values,std.values):
    err_x.append((x,x))
    err_y.append((y-yerr,y+yerr))

s_color = 'coral'    
s_width = 1.5
p.rect(mean.index+1, mean/2, width=.3, height=mean,
      fill_color=None, line_color=s_color, line_width=s_width,
      legend='Mean',
       source=source2)
p.multi_line(err_x,err_y,
             color=s_color,line_width=s_width,
             legend='Stdv')

p.xgrid.grid_line_color = None
p.xaxis.major_tick_line_color = None
# p.legend.location('top_right')

output_file('images/day_inspection.html')
show(p)

ERROR:C:\Anaconda\lib\site-packages\bokeh\core\validation\check.pyc:E-1000 (COLUMN_LENGTHS): ColumnDataSource column lengths are not all the same: ColumnDataSource, ViewModel:ColumnDataSource, ref _id: 00b67471-15d9-456c-8ae9-25587d699a62
ERROR:C:\Anaconda\lib\site-packages\bokeh\core\validation\check.pyc:E-1000 (COLUMN_LENGTHS): ColumnDataSource column lengths are not all the same: ColumnDataSource, ViewModel:ColumnDataSource, ref _id: ac0bb640-1e0c-4494-94e7-837847bccd90
ERROR:C:\Anaconda\lib\site-packages\bokeh\core\validation\check.pyc:E-1000 (COLUMN_LENGTHS): ColumnDataSource column lengths are not all the same: ColumnDataSource, ViewModel:ColumnDataSource, ref _id: ed775fa5-7b5c-4236-af27-411fd8dad73e
ERROR:C:\Anaconda\lib\site-packages\bokeh\core\validation\check.pyc:E-1000 (COLUMN_LENGTHS): ColumnDataSource column lengths are not all the same: ColumnDataSource, ViewModel:ColumnDataSource, ref _id: 09769417-0a59-4686-9c56-d84cfc727a45
ERROR:C:\Anaconda\lib\site-packages\bokeh\co

# Score by day

In [401]:
score_day = df15.groupby(['INSPECTION DATE','CAMIS'])['SCORE'].max()

In [404]:
score_day = score_day.mean(level=0)

In [411]:
day_2015 = pd.concat([day_2015, score_day], axis=1)

In [412]:
day_2015.head()

,INSPECTIONS,DAY,SCORE
2015-01-02,142,4,12.601449
2015-01-03,26,5,15.791667
2015-01-04,0,6,NaN
2015-01-05,201,0,13.994975
2015-01-06,205,1,14.875622


In [421]:
type(day_2015['SCORE'])

pandas.core.series.Series

In [444]:
from bokeh.models import Span

p = figure(title = 'NYC inspection score by day',
           x_range=days_of_week, y_range=Range1d(0,25))
p.title_text_font_size = '14pt'
# p.xaxis.axis_label_text_font_size = '10pt'

hover = HoverTool(tooltips='@DATE')
p.add_tools(hover)

source = ColumnDataSource(day_2015)
source.add(day_2015.index.strftime('%m/%d/%y'), name="DATE")

# A-B line
AB_threshold = Span(location=13.5, dimension='width',
                   line_color='#BBBBBB', line_dash='dashed')
p.renderers.extend([AB_threshold])

#def mtext(x,y,color):
#    p.text(0.8,12, text='[A]', text_font_style='bold', text_font_size,  text_color='royalblue')

x_pos = 7.3
f_size = '11pt'
p.text(x_pos,12.5, text=['A'], text_font_style='bold', text_font_size=f_size,  text_color='royalblue')
p.text(x_pos,13.7, text=['B'], text_font_style='bold', text_font_size=f_size, text_color='forestgreen')

c_alpha = 0.5
p.circle(day_2015['DAY']+1,'SCORE',
         # line_alpha=c_alpha,
         fill_alpha=c_alpha,
         line_width=1,
         line_color=None,
         size = 6,
         legend = 'Date(s)',
         source = source)

mean = day_2015.groupby("DAY")['SCORE'].mean()
std = day_2015.groupby("DAY")['SCORE'].std()
source2 = ColumnDataSource({})
source2.add(mean.values,name="DATE") 

err_x = []
err_y = []

for x, y, yerr in zip(mean.index+1,mean.values,std.values):
    err_x.append((x,x))
    err_y.append((y-yerr,y+yerr))

s_color = 'coral'    
s_width = 1.5
p.rect(mean.index+1, mean/2, width=.3, height=mean,
      fill_color=None, line_color=s_color, line_width=s_width,
      legend='Mean',
       source=source2)
p.multi_line(err_x,err_y,
             color=s_color,line_width=s_width,
             legend='Stdv')

p.xgrid.grid_line_color = None
p.xaxis.major_tick_line_color = None

output_file('images/score_day.html')
show(p)

ERROR:C:\Anaconda\lib\site-packages\bokeh\core\validation\check.pyc:E-1000 (COLUMN_LENGTHS): ColumnDataSource column lengths are not all the same: ColumnDataSource, ViewModel:ColumnDataSource, ref _id: 00b67471-15d9-456c-8ae9-25587d699a62
ERROR:C:\Anaconda\lib\site-packages\bokeh\core\validation\check.pyc:E-1000 (COLUMN_LENGTHS): ColumnDataSource column lengths are not all the same: ColumnDataSource, ViewModel:ColumnDataSource, ref _id: ac0bb640-1e0c-4494-94e7-837847bccd90
ERROR:C:\Anaconda\lib\site-packages\bokeh\core\validation\check.pyc:E-1000 (COLUMN_LENGTHS): ColumnDataSource column lengths are not all the same: ColumnDataSource, ViewModel:ColumnDataSource, ref _id: ed775fa5-7b5c-4236-af27-411fd8dad73e
ERROR:C:\Anaconda\lib\site-packages\bokeh\core\validation\check.pyc:E-1000 (COLUMN_LENGTHS): ColumnDataSource column lengths are not all the same: ColumnDataSource, ViewModel:ColumnDataSource, ref _id: 09769417-0a59-4686-9c56-d84cfc727a45
ERROR:C:\Anaconda\lib\site-packages\bokeh\co

# violation dict

In [31]:
query = ('https://data.cityofnewyork.us/resource/9w7m-hzhe.json?'
    '$select=violation_code,violation_description'
    '&$where=inspection_date>="2015-01-01"'
    '&$limit=200000')
violation_df = pd.read_json(query)

In [32]:
violation_description = violation_df.groupby(violation_df['violation_code'])['violation_description'].unique()
for i in range(len(violation_description)):
    violation_description.iloc[i] = violation_description.iloc[i][0]

In [38]:
dict_violation = violation_description.to_dict()

import pickle
with open('vars.pickle','wb') as f:
    pickle.dump(dict_violation, f) 

In [109]:
query = ('https://data.cityofnewyork.us/resource/9w7m-hzhe.json?'
    '$select=violation_code,violation_description'
    '&$where=inspection_date>="2015-01-01"'
    '&$limit=5')
violation_df2 = pd.read_json(query)

In [111]:
violation_df.iloc[0,1]

u'Filth flies or food/refuse/sewage-associated (FRSA) flies present in facility\x1as food and/or non-food areas. Filth flies include house flies, little house flies, blow flies, bottle flies and flesh flies. Food/refuse/sewage-associated flies include fruit flies, drain flies and Phorid flies.'

# Test

In [14]:
d = {'a':[1,1,2,3,1],
     'b':[1,2,2,1,2],
    'c':['x','y','z','y','x']}
dfx = pd.DataFrame(d)

In [29]:
a = dfx.groupby(['a','b'])['c'].unique()

In [92]:
dfx

,a,b,c
0,1,1,x
1,1,2,y
2,2,2,z
3,3,1,y
4,1,2,x


In [102]:
dfx.query('len(a)==1')

ValueError: "len" is not a supported function

In [97]:
df3 = pd.DataFrame({'A': [1, 2, 3],
    'B': [4, 5, 6],
    'C': [7, 8, 9]})
df3.where(lambda x: x > 4, lambda x: x + 10)

ValueError: where requires an ndarray like object for its condition

In [100]:
pd.__version__

u'0.18.0'

In [345]:
freq = dfx.groupby(['a','b']).count()

In [346]:
freq.loc[1,1]

c    1
Name: (1, 1), dtype: int64

In [347]:
freq

c
a b   
1 1  1
  2  2
2 2  1
3 1  1

In [350]:
dfx.apply(lambda x: freq.loc[x['a'],x['b']], axis=1)

,c
0,1
1,2
2,1
3,1
4,2


In [371]:
d = {'a':[1,1,2,3,1],
     'b':[1,2,2,1,2],
     'c':['x','x','x','x','x']}
df1 = pd.DataFrame(d)
df1.groupby(['a','b']).count() 

c
a b   
1 1  1
  2  2
2 2  1
3 1  1

In [376]:
df2 = df1.drop('c',1)
q = df2.groupby(['a'])['b'].value_counts() 

In [377]:
type(q)

pandas.core.series.Series

In [378]:
q

a  b
1  2    2
   1    1
2  2    1
3  1    1
dtype: int64

In [379]:
q[1,2]

2

In [383]:
dft.head()

DAY  INSPECTIONS
0    191            4
     164            3
     168            3
     196            3
     208            3
dtype: int64

In [384]:
dft[0,191]

4